# Day 5: Integrazione con Azure Data Lake Gen2

## Differenza tra Catalogo, Schema e Volume in Databricks

### Gerarchia e definizioni

- **Volume**: È un contenitore di archiviazione progettato specificamente per dati non strutturati (come file CSV, JSON, immagini, log, ecc.) e risiede all'interno di uno schema. I volumi sono "fratelli" delle tabelle e delle viste all'interno di uno schema in Unity Catalog. I volumi consentono un accesso governato ai dati non tabellari in Databricks.

| Concetto | Livello Gerarchico | Tipo di Dati | Scopo Principale |
|--------|-------------------|--------------|------------------|
| **Catalogo** | Livello più alto | Tutti i tipi | Isolamento organizzativo e governance aziendale |
| **Schema** | Livello intermedio (dentro un catalogo) | Strutturati e non strutturati | Organizzazione logica dei dati (es. per dominio) |
| **Volume** | Oggetto all'interno di uno schema | Non strutturati (file) | Gestione governata di file e dati grezzi |

### Esempio pratico

```sql
-- Creazione di un catalogo per il team analisi
CREATE CATALOG analytics;

-- Creazione di uno schema per i dati marketing
CREATE SCHEMA analytics.marketing;

-- Creazione di un volume per file CSV e JSON
CREATE VOLUME analytics.marketing.raw_data;

-- Creazione di una tabella per dati strutturati
CREATE TABLE analytics.marketing.campaigns (...);

## Cos'è Azure Data Lake Storage Gen2 (ADLS Gen2)

Azure Data Lake Storage Gen2 è un servizio di archiviazione dati su larga scala che combina le funzionalità di Azure Data Lake Storage Gen1 con la scalabilità e l'economia di Azure Blob Storage, offrendo un'architettura ottimizzata per carichi di lavoro analitici.

---

## Caratteristiche Principali

- **Namespace Gerarchico**: Implementa una struttura a directory e file compatibile con Hadoop, essenziale per le applicazioni big data.
- **Sicurezza Enterprise**: Include funzionalità di sicurezza integrate come Microsoft Entra ID (precedentemente Azure AD) e liste di controllo accesso POSIX (ACLs).
- **Scalabilità Massiva**: Progettato per gestire petabyte di dati con prestazioni elevate.
- **Supporto Ecosistema Big Data**: Compatibile con strumenti come Azure Databricks, HDInsight e altri servizi Azure.
- **Costi Ottimizzati**: Combina i vantaggi economici di Azure Blob Storage con le funzionalità avanzate per l'analisi dati.

> 💡 **Note**:  
> - ADLS Gen2 si basa su Azure Blob Storage ma aggiunge un namespace gerarchico per una gestione più semplice dei dati.  
> - È ideale per scenari di data lake, data warehouse e analytics scalabili.  
> - Supporta sia dati strutturati che non strutturati (CSV, JSON, Parquet, ecc.).

# 🌐 Integrazione Azure Data Lake Storage Gen2 e Azure Databricks

Questo notebook illustra i metodi principali per accedere ai dati in **Azure Data Lake Storage Gen2 (ADLS Gen2)** da **Azure Databricks**, con focus su sicurezza, praticità e best practice.

---

## 🔹 1. Montaggio del Filesystem (Mounting)

Il metodo più comodo per accedere ai dati come se fossero in una directory locale.

### ✅ Vantaggi
- Accesso diretto e semplice
- I percorsi diventano `/mnt/<nome-mount>`
- Compatibile con tutti i formati (Parquet, CSV, JSON, ecc.)

### ⚠️ Svantaggi
- Richiede gestione delle credenziali
- Non ideale per ambienti multi-tenant o produzione avanzata

### 🔧 Passaggi

1. **Salvare la chiave di accesso in un secret scope**
   ```python
   dbutils.secrets.put(scope="talendsadlsgen2", key="storage_account_key", value="<your-storage-key>")